# Dump Models to Excel

## Libraries

In [5]:
import os
import datetime
import traceback
import logging
import sqlite3
import sys

from logging.handlers import TimedRotatingFileHandler
from dotenv import load_dotenv
from sqlalchemy import event
from sqlalchemy import create_engine

import pandas as pd

## Environment Setting

In [6]:
fhandler = TimedRotatingFileHandler("logs/dump_model.log", when="midnight", interval=1)
fhandler.suffix = "%Y%m%d"
logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[fhandler, logging.StreamHandler(sys.stdout)],
    )
logger = logging.getLogger(__name__)

database_folder = "database"
database_name = f"{database_folder}/main.db"

load_dotenv()

use_sqlite = eval(os.getenv('USE_SQLITE3'))

excel_file = "excelfiles/{0}.csv"

## Connecting to Database

In [7]:
my_conn = None
if use_sqlite:
  my_conn = sqlite3.connect(database_name)
else:
  my_conn = create_engine(os.getenv("PG_DATA_CONN"))  

## Routine

In [8]:
SQL_TABLES = ""
if use_sqlite:
  SQL_TABLES = "select * from sqlite_schema"
else:
  SQL_TABLES = "select table_name as name from information_schema.tables WHERE table_schema = 'main' AND table_type = 'BASE TABLE'"

tables = pd.read_sql(SQL_TABLES, my_conn)

for idx in range(tables['name'].size):
  logger.info(f"Reading table {tables['name'][idx]}...")
  table_data = pd.read_sql(f"select * from main.{tables['name'][idx]}", my_conn)
  if table_data.size > 0:
    logger.info(f"Saving table {tables['name'][idx]} to file...")
    table_data.to_csv(excel_file.format(tables['name'][idx]))


2022-10-26 18:54:03,955 - INFO - Reading table fact_case_conclusions...
2022-10-26 18:54:04,394 - INFO - Saving table fact_case_conclusions to file...
2022-10-26 18:54:05,134 - INFO - Reading table fact_cases...
2022-10-26 18:57:03,894 - INFO - Saving table fact_cases to file...
2022-10-26 19:00:50,238 - INFO - Reading table fact_epa_informal_actions...
2022-10-26 19:01:00,878 - INFO - Saving table fact_epa_informal_actions to file...
2022-10-26 19:01:00,999 - INFO - Reading table fact_programs...
2022-10-26 19:01:10,697 - INFO - Saving table fact_programs to file...
2022-10-26 19:01:25,472 - INFO - Reading table fact_wifia_letter...
2022-10-26 19:01:26,279 - INFO - Saving table fact_wifia_letter to file...
